In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import warnings
import concurrent.futures
import string
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:

def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    return soup

def extract_links(url):
    soup = get_soup(url)
    return [h3.a["href"] for h3 in soup.find_all(name="h3", class_="product-name")]
all_links1 = [f"https://donghohaitrieu.com/danh-muc/dong-ho-nu/page/{idx_page}" for idx_page in range(1, 49)]
all_links2 = [f"https://donghohaitrieu.com/danh-muc/dong-ho-nam/page/{idx_page}" for idx_page in range(1, 57)]
all_links3 = [f"https://donghohaitrieu.com/danh-muc/dong-ho-doi-cap/page/{idx_page}" for idx_page in range(1, 2)]
all_links = []


with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(extract_links, url) for url in all_links1 + all_links2 + all_links3]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

all_links = [link for sublist in results for link in sublist]



In [3]:
all_links = list(set(all_links))
all_links

['https://donghohaitrieu.com/san-pham/tissot-t063-209-11-038-00-nu-kinh-sapphire-quartz-pin-day-kim-loai-mat-so-31mm',
 'https://donghohaitrieu.com/san-pham/bulova-96p198-nu-kinh-sapphire-quartz-pin-day-kim-loai',
 'https://donghohaitrieu.com/san-pham/casio-a100we-7bdf-nam-kinh-nhua-quartz-pin-day-kim-loai',
 'https://donghohaitrieu.com/san-pham/casio-efv-540d-1avudf-nam-quartz-pin-day-kim-loai',
 'https://donghohaitrieu.com/san-pham/casio-w-218hc-4a2vdf-nu-kinh-nhua-quartz-pin-day-cao-su-mat-so-43-2mm',
 'https://donghohaitrieu.com/san-pham/longines-l4-321-4-92-6-nu-kinh-sapphire-automatic-tu-dong-day-kim-loai',
 'https://donghohaitrieu.com/san-pham/doxa-d155rcm-nam-kinh-sapphire-quartz-pin-day-kim-loai',
 'https://donghohaitrieu.com/san-pham/daniel-wellington-0205dw-nam-quartz-pin-day-vai',
 'https://donghohaitrieu.com/san-pham/daniel-wellington-dw00100436-nam-quartz-pin-day-cao-su-mat-so-40mm',
 'https://donghohaitrieu.com/san-pham/certina-c033-450-16-031-00-nam-kinh-sapphire-quartz

In [4]:
len(all_links)

2488

In [5]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup

def get_product_info(url):
    soup = get_soup(url)
    price = soup.find_all("p", class_="price")[0].text.replace("\xa0₫", "").strip()
    details = {}
    content = soup.find("div", {"class": "vc_row wpb_row vc_row-fluid"})
    table = content.find("div", {"class": "wpb_wrapper"})
    rows = table.find_all(["p", "tr"])
    for row in rows:
        text = (
            row.get_text()
            .replace("\n", "")
            .replace("(Tùy từng lô hàng)", "")
            
        )
        if ":" in text:
            if len(text.split(":")) == 2:
                key, value = text.split(":")
                details[key.lower().strip()] = value.strip()
        elif "\xa0" in text:
            if len(text.split("\xa0")) == 2:
                key, value = text.replace(":", "").split("\xa0")
                details[key.lower().strip()] = value.strip()
    name = (
        soup.find("h1", {"class": "product_title"})
        .text.strip()
        .split(" ")[0]
        .replace(")", "")
        .replace("(", "")
        )
    a = {"Tên sản phẩm": name, "Giá tiền": price, **details}
    print(a)
    return {"Tên sản phẩm": name, "Giá tiền": price, **details}


In [6]:
def crawl_data_from_web(all_links):
    data_frame_watchs = pd.DataFrame()
    with ThreadPoolExecutor(max_workers=15) as executor:
        results = executor.map(get_product_info, all_links)
        for result in results:
            data_frame_watchs = data_frame_watchs.append(result, ignore_index=True)
    return data_frame_watchs



In [7]:
data_frame_watchs = crawl_data_from_web(all_links)

{'Tên sản phẩm': 'Casio', 'Giá tiền': '803.000', 'thương hiệu': 'Casio', 'số hiệu sản phẩm': 'W-218HC-4A2VDF', 'xuất xứ': 'Nhật Bản', 'giới tính': 'Nữ', 'kính': 'Resin Glass (Kính Nhựa)', 'máy': 'Quartz (Pin)', 'bảo hành quốc tế': '1 Năm', 'bảo hành tại hải triều': '', 'đường kính mặt số': '44.4 mm x 43.2 mm', 'bề dày mặt số': '11 mm', 'niềng': 'Nhựa', 'dây đeo': 'Dây Cao Su', 'màu mặt số': 'Trắng', 'chống nước': '5 ATM', 'chức năng': 'Lịch – Bộ Bấm Giờ – Giờ Kép – Đèn Led – Vài Chức Năng Khác', 'nơi sản xuất': 'Trung Quốc'}
{'Tên sản phẩm': 'Casio', 'Giá tiền': '3.529.000', 'thương hiệu': 'Casio', 'số hiệu sản phẩm': 'EFV-540D-1AVUDF', 'xuất xứ': 'Nhật Bản', 'giới tính': 'Nam', 'kính': 'Mineral Crystal (Kính Cứng)', 'máy': 'Quartz (Pin)', 'bảo hành quốc tế': '1 Năm', 'bảo hành tại hải triều': '', 'đường kính mặt số': '43.8 mm', 'bề dày mặt số': '12.1 mm', 'niềng': '', 'dây đeo': '', 'màu mặt số': 'Đen', 'chống nước': '10 ATM', 'chức năng': 'Lịch Ngày – Chronograph', 'nơi sản xuất': 'T

In [8]:
data_frame_watchs.isnull().sum()

Tên sản phẩm                                    0
Giá tiền                                        0
thương hiệu                                     6
số hiệu sản phẩm                                3
xuất xứ                                         3
giới tính                                      39
kính                                            3
máy                                             3
bảo hành quốc tế                                2
bảo hành tại hải triều                        233
đường kính mặt số                               3
bề dày mặt số                                  29
niềng                                         134
dây đeo                                        79
màu mặt số                                      3
chống nước                                      2
chức năng                                     389
nơi sản xuất                                  701
thông số đặc biệt                            2207
bộ sưu tập                                   2422


In [9]:
data_frame_watchs

,Tên sản phẩm,Giá tiền,thương hiệu,số hiệu sản phẩm,xuất xứ,giới tính,kính,máy,bảo hành quốc tế,bảo hành tại hải triều,...,bل؛£o hأ nh quل»‘c tل؛؟,bل؛£o hأ nh tل؛،i hل؛£i triل»پu,ؤگئ°ل»‌ng kأ­nh mل؛·t sل»‘,bل»پ dأ y mل؛·t sل»‘,niل»پng,dأ¢y ؤگeo,mأ u mل؛·t sل»‘,chل»‘ng nئ°ل»›c,chل»©c nؤƒng,nئ،i sل؛£n xuل؛¥t (tأ¹y tل»«ng lأ´ hأ ng)
0,Tissot,12.250.000,Tissot,T063.209.11.038.00,Thụy Sỹ,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),2 Năm,4 Năm – RED Guarantee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bulova,8.500.000,Bulova,96P198,Thụy Sỹ / Mỹ,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),3 Năm,4 Năm – RED Guarantee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Casio,1.633.000,Casio,A100WE-7BDF,Nhật Bản,Nam,Resin Glass (Kính Nhựa),Quartz (Pin),1 Năm,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Casio,3.529.000,Casio,EFV-540D-1AVUDF,Nhật Bản,Nam,Mineral Crystal (Kính Cứng),Quartz (Pin),1 Năm,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Casio,803.000,Casio,W-218HC-4A2VDF,Nhật Bản,Nữ,Resin Glass (Kính Nhựa),Quartz (Pin),1 Năm,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,Frederique,58.510.000,Frederique Constant,FC-306G4S6B,Thụy Sỹ,Nam,Sapphire (Kính Chống Trầy),Quartz (Pin)/Automatic (Tự Động),2 Năm,4 Năm – RED Guarantee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,Seiko,6.170.000,Seiko,SUR463P1,Nhật Bản,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),1 Năm,5 Năm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,Daniel,4.940.000,Daniel Wellington,DW00100437,Thụy Điển,Nữ,Mineral Crystal (Kính Cứng),Quartz (Pin),2 Năm,5 Năm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,Citizen,3.470.000,Citizen,EV0052-50E,Nhật Bản,Nữ,Mineral Crystal (Kính Cứng),Quartz (Pin),1 Năm,5 Năm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data_frame_watchs = data_frame_watchs.dropna(axis=1, thresh=len(data_frame_watchs)*0.4)
data_frame_watchs

,Tên sản phẩm,Giá tiền,thương hiệu,số hiệu sản phẩm,xuất xứ,giới tính,kính,máy,bảo hành quốc tế,bảo hành tại hải triều,đường kính mặt số,bề dày mặt số,niềng,dây đeo,màu mặt số,chống nước,chức năng,nơi sản xuất
0,Tissot,12.250.000,Tissot,T063.209.11.038.00,Thụy Sỹ,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),2 Năm,4 Năm – RED Guarantee,31 mm,5.2 mm,Thép Không Gỉ,Thép Không Gỉ,Trắng,3 ATM,,Thụy Sỹ
1,Bulova,8.500.000,Bulova,96P198,Thụy Sỹ / Mỹ,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),3 Năm,4 Năm – RED Guarantee,32 mm,8 mm,Thép Không Gỉ,Thép Không Gỉ,Xanh Đen,3 ATM,NaN,Thụy Sỹ
2,Casio,1.633.000,Casio,A100WE-7BDF,Nhật Bản,Nam,Resin Glass (Kính Nhựa),Quartz (Pin),1 Năm,,40.7 mm x 32.7 mm,9.2 mm,,,Đen,3 ATM,Lịch – Bộ Bấm Giờ – Giờ Kép – Vài Chức Năng Khác,Trung Quốc
3,Casio,3.529.000,Casio,EFV-540D-1AVUDF,Nhật Bản,Nam,Mineral Crystal (Kính Cứng),Quartz (Pin),1 Năm,,43.8 mm,12.1 mm,,,Đen,10 ATM,Lịch Ngày – Chronograph,Trung Quốc
4,Casio,803.000,Casio,W-218HC-4A2VDF,Nhật Bản,Nữ,Resin Glass (Kính Nhựa),Quartz (Pin),1 Năm,,44.4 mm x 43.2 mm,11 mm,Nhựa,Dây Cao Su,Trắng,5 ATM,Lịch – Bộ Bấm Giờ – Giờ Kép – Đèn Led – Vài Ch...,Trung Quốc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,Frederique,58.510.000,Frederique Constant,FC-306G4S6B,Thụy Sỹ,Nam,Sapphire (Kính Chống Trầy),Quartz (Pin)/Automatic (Tự Động),2 Năm,4 Năm – RED Guarantee,40 mm,8 mm,Thép Không Gỉ,Thép Không Gỉ,Đen,3 ATM,Lịch Ngày,Thụy Sỹ
2484,Seiko,6.170.000,Seiko,SUR463P1,Nhật Bản,Nữ,Sapphire (Kính Chống Trầy),Quartz (Pin),1 Năm,5 Năm,29.8 mm,8.3 mm,Thép Không Gỉ,Thép Không Gỉ,Trắng,10 ATM,Lịch Ngày,NaN
2485,Daniel,4.940.000,Daniel Wellington,DW00100437,Thụy Điển,Nữ,Mineral Crystal (Kính Cứng),Quartz (Pin),2 Năm,5 Năm,20 mm x 26 mm,6 mm,Thép Không Gỉ,Thép Không Gỉ,Xanh,3 ATM,,Trung Quốc
2486,Citizen,3.470.000,Citizen,EV0052-50E,Nhật Bản,Nữ,Mineral Crystal (Kính Cứng),Quartz (Pin),1 Năm,5 Năm,35 mm,9 mm,Thép Không Gỉ,Thép Không Gỉ,Đen,3 ATM,Lịch Ngày,Trung Quốc


In [11]:

data_frame_watchs.to_csv("haitrieu.csv")